In [1]:
from bpch2nc import bpch_2_netcdf
import numpy as np
import xarray as xr
import re

In [2]:
case_name = 'CO2-2018'
first_day   = '2018-01-01'
last_day_p1 = '2018-02-01'

In [3]:
# Name of Bpch file
directory = '/geos/u73/msadiq/GEOS-Chem/rundirs/ensemble_runs/' + case_name + '/nd51/'

name_bpch1 = 'ts_satellite.'
tinfo_file = directory + 'tracerinfo.dat'
dinfo_file = directory + 'diaginfo.dat'

# Output
output_directory = '/geos/u73/msadiq/GEOS-Chem/rundirs/ensemble_runs/' + case_name + '/nd51/'

# Number of seconds in the diagnostic interval (assume 1-month)
# does not matter for CO2
interval = 86400.0 * 31.0

In [4]:
days = np.arange(first_day, last_day_p1, dtype='datetime64[D]')

for iday in np.arange(len(days)):
    day_string = days[iday] # format not right for the following function
    print('converting bpch to netcdf on day: ', day_string)
    new_day_string = re.sub("[^0-9]", "", str(day_string)) # strip off '-'s
    
    bpchfile = directory + name_bpch1 + new_day_string + '.bpch'
    ncfile = output_directory + name_bpch1 + new_day_string + '.nc'
    
    bpch_2_netcdf(bpchfile=bpchfile, 
                  tinfo_file=tinfo_file, 
                  dinfo_file=dinfo_file, 
                  ncfile=ncfile)

converting bpch to netcdf on day:  2018-01-01
converting bpch to netcdf on day:  2018-01-02
converting bpch to netcdf on day:  2018-01-03
converting bpch to netcdf on day:  2018-01-04
converting bpch to netcdf on day:  2018-01-05
converting bpch to netcdf on day:  2018-01-06
converting bpch to netcdf on day:  2018-01-07
converting bpch to netcdf on day:  2018-01-08
converting bpch to netcdf on day:  2018-01-09
converting bpch to netcdf on day:  2018-01-10
converting bpch to netcdf on day:  2018-01-11
converting bpch to netcdf on day:  2018-01-12
converting bpch to netcdf on day:  2018-01-13
converting bpch to netcdf on day:  2018-01-14
converting bpch to netcdf on day:  2018-01-15
converting bpch to netcdf on day:  2018-01-16
converting bpch to netcdf on day:  2018-01-17
converting bpch to netcdf on day:  2018-01-18
converting bpch to netcdf on day:  2018-01-19
converting bpch to netcdf on day:  2018-01-20
converting bpch to netcdf on day:  2018-01-21
converting bpch to netcdf on day: 

In [5]:
# prepare output data format
new_day_string = re.sub("[^0-9]", "", str(first_day)) # strip off '-'s
first_file = xr.open_dataset(directory + name_bpch1 + new_day_string + '.nc')
varnames = list(first_file.data_vars.keys())  # a list of variable names
lon = first_file.lon
lat = first_file.lat
lev = first_file.lev
time = days
target = xr.DataArray(np.nan, coords=[time, lev, lat, lon], dims=['time', 'lev', 'lat', 'lon'])
output = target.to_dataset(name = 'SpeciesConc_CO2')
output.attrs = first_file.attrs
for ivar in varnames: output[ivar] = target.copy()


In [6]:
# combine the netcdf files into one, monthly
for iday in np.arange(len(days)):
    day_string = days[iday] #
    print(day_string)
    new_day_string = re.sub("[^0-9]", "", str(day_string)) # strip off '-'s
    ncfile = output_directory + name_bpch1 + new_day_string + '.nc'
    
    ds_tmp = xr.open_dataset(ncfile)
    
    for ivar in varnames:
        output[ivar][iday,:,:,:] = ds_tmp[ivar][0,:,:,:].copy()
        output[ivar].attrs = ds_tmp[ivar].attrs

2018-01-01
2018-01-02
2018-01-03
2018-01-04
2018-01-05
2018-01-06
2018-01-07
2018-01-08
2018-01-09
2018-01-10
2018-01-11
2018-01-12
2018-01-13
2018-01-14
2018-01-15
2018-01-16
2018-01-17
2018-01-18
2018-01-19
2018-01-20
2018-01-21
2018-01-22
2018-01-23
2018-01-24
2018-01-25
2018-01-26
2018-01-27
2018-01-28
2018-01-29
2018-01-30
2018-01-31


In [7]:
# output monthly combined data into a NetCDF file
first_day_string = re.sub("[^0-9]", "", str(first_day)) # strip off '-'s
monthly_string = first_day_string[0:6]
output.to_netcdf(output_directory + name_bpch1 + monthly_string + '.nc')

In [8]:
# flatten the 4d output
ds = xr.open_dataset(output_directory + name_bpch1 + monthly_string + '.nc')
ds['SpeciesConc_CO2']

<xarray.DataArray 'SpeciesConc_CO2' (time: 31, lev: 47, lat: 91, lon: 144)>
[19092528 values with dtype=float64]
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-31
  * lev      (lev) float64 0.9925 0.9775 0.9624 ... 0.0003988 0.000127 2.763e-05
  * lat      (lat) float64 -89.5 -88.0 -86.0 -84.0 -82.0 ... 84.0 86.0 88.0 89.5
  * lon      (lon) float64 -180.0 -177.5 -175.0 -172.5 ... 172.5 175.0 177.5
Attributes:
    units:      ppbv
    long_name:  CO2 tracer

In [9]:
nm_lev = len(ds.lev)
nm_lon = len(ds.lon)
nm_lat = len(ds.lat)
nm_days = len(ds.time)

record = range(nm_lon * nm_lat * nm_days)
record

range(0, 406224)

In [10]:
target = xr.DataArray(np.nan, coords=[record, ds.lev], dims=['record', 'levels'])
ds_output = target.to_dataset(name = 'SpeciesConc_CO2')
for ivar in varnames: ds_output[ivar] = target.copy()
ds_output

<xarray.Dataset>
Dimensions:          (levels: 47, record: 406224)
Coordinates:
  * record           (record) int64 0 1 2 3 4 ... 406220 406221 406222 406223
  * levels           (levels) float64 0.9925 0.9775 ... 0.000127 2.763e-05
Data variables:
    SpeciesConc_CO2  (record, levels) float64 nan nan nan nan ... nan nan nan
    PEDGE_S_PSURF    (record, levels) float64 nan nan nan nan ... nan nan nan

In [11]:
flat = ds['SpeciesConc_CO2'][:,0,:,:].to_dataframe() # flatten a variable at one level
flat.reset_index(inplace=True) # get indices to prepare output coordinates
flat

,time,lat,lon,lev,SpeciesConc_CO2
0,2018-01-01,-89.5,-180.0,0.9925,403691.09375
1,2018-01-01,-89.5,-177.5,0.9925,403693.31250
2,2018-01-01,-89.5,-175.0,0.9925,403695.84375
3,2018-01-01,-89.5,-172.5,0.9925,403697.75000
4,2018-01-01,-89.5,-170.0,0.9925,403699.93750
...,...,...,...,...,...
406219,2018-01-31,89.5,167.5,0.9925,411174.18750
406220,2018-01-31,89.5,170.0,0.9925,411173.65625
406221,2018-01-31,89.5,172.5,0.9925,411173.50000
406222,2018-01-31,89.5,175.0,0.9925,411173.18750


In [12]:
lat = xr.DataArray(0, coords=[record], dims=['record'])
lon = xr.DataArray(0, coords=[record], dims=['record'])
date = xr.DataArray(0, coords=[record], dims=['record'])
lat.values = flat['lat']
lon.values = flat['lon']
date.values = flat['time']
ds_output['lat'] = lat
ds_output['lon'] = lon
ds_output['date'] = date
ds_output

<xarray.Dataset>
Dimensions:          (levels: 47, record: 406224)
Coordinates:
  * record           (record) int64 0 1 2 3 4 ... 406220 406221 406222 406223
  * levels           (levels) float64 0.9925 0.9775 ... 0.000127 2.763e-05
Data variables:
    SpeciesConc_CO2  (record, levels) float64 nan nan nan nan ... nan nan nan
    PEDGE_S_PSURF    (record, levels) float64 nan nan nan nan ... nan nan nan
    lat              (record) float64 -89.5 -89.5 -89.5 -89.5 ... 89.5 89.5 89.5
    lon              (record) float64 -180.0 -177.5 -175.0 ... 172.5 175.0 177.5
    date             (record) datetime64[ns] 2018-01-01 ... 2018-01-31

In [13]:
for ivar in varnames:
    target = xr.DataArray(np.nan, coords=[record, ds.lev], dims=['record', 'levels'])
    for ilev in range(nm_lev):
        flat = ds[ivar][:,ilev,:,:].to_dataframe() # flatten a variable at one level
        target[:,ilev] = flat[ivar] # store output to a dataarray
    ds_output[ivar] = target.copy() # store dataarray to dataset
    print(ivar + ' done!')

SpeciesConc_CO2 done!
PEDGE_S_PSURF done!


In [14]:
# 
ds_output.attrs['comment'] = '[CO2] unit: ppbv'
ds_output.record.values

array([     0,      1,      2, ..., 406221, 406222, 406223])

In [15]:
output_file = '2d_' + name_bpch1 + monthly_string + '.nc'
ds_output.to_netcdf(output_directory + output_file)
print(output_file)

2d_ts_satellite.201801.nc
